## Stitching Networks between Chameleon and an ExoGENI Stitchport

An example that can be used to stitch to an ExoGENI Stitchport

## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

#Config with your project and site
chi.set('project_name', 'chameleon')   # Replace with your project name
chi.use_site('CHI@UC')                 # Authenticate to CHI@UC site

#GENI Pem file
geni_pem_file = '/home/pruth/work/geni-pruth.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_ExoGENIStitchportTutorial_"
server_name = prefix+'Server'
network_name = prefix+'Net'
stitched_network_name = network_name+"Stitched"
subnet_name = prefix+'Subnet'
router_name = prefix+'Router'
lease_name = prefix+'Lease'
lease_name_network = prefix+'LeaseNet'
lease_name_servers = prefix+'LeaseServers'
lease_name_stitch = prefix+'LeaseStitch'


#Server Config
image_name = 'CC-Ubuntu20.04'
node_type = "compute_skylake"
server_count = 1

#optionally set the OpenFlow controller (set ot None for non-OpenFlow swtich)
of_controller_ip=None
#of_controller_ip = "192.5.87.215"
#of_controller_port = "6653"


#Set the name of the VFC
vswitch_name = "pruthSDN"

#Stitchport URL
stitchport_url = ""    # Your stitchport URL
stitchport_vlan = ""   # Your stitchport VLAN

## Create the Network (OpenFlow Optional)

#### Create a Lease at Chicago

In [ ]:
import chi.lease
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []

if of_controller_ip:
    print("OpenFlow")
    chi.lease.add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            of_controller_ip=of_controller_ip, 
                            of_controller_port=of_controller_port, 
                            vswitch_name=vswitch_name)
else:
    print("No OpenFlow")
    chi.lease.add_network_reservation(reservation_list, 
                            network_name=network_name, 
                            physical_network='physnet1',
                            vswitch_name=vswitch_name)
                           

# Create the lease
network_lease = chi.lease.create_lease(lease_name,
                       reservations=reservation_list,
                       start_date=start_date,
                       end_date=end_date)

network_lease = chi.lease.wait_for_active(network_lease['id'])
    
#Print the lease info
print(json.dumps(network_lease, indent=2))

#### Get the Nework Reservation

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can get the leaseand separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
network_reservation_id = [reservation for reservation in network_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']

print("network_reservation_id: " + network_reservation_id)

In [ ]:
import chi.network

#Get the network
network = chi.network.get_network(network_name)

#Get the network ID
network_id = network['id']
print(f'Network ID: {network_id}')

#Get the VLAN tag (needed for ExoGENI stitching)
network_vlan = network['provider:segmentation_id']
print(f'network_vlan: {network_vlan}')

#### Add a subnet

Adds a subnet to the reserved network. 

In [ ]:
subnet = chi.network.create_subnet(subnet_name, network_id)

print(json.dumps(subnet, indent=2))

#### Add a Router

In [ ]:
router = chi.network.create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

#### Attach the Router and Subnet

In [ ]:
chi.network.add_subnet_to_router_by_name(router_name, subnet_name)

#### Add a Circuit Stitched to ExoGENI

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []

chi.lease.add_network_reservation(reservation_list,
                                  network_name=stitched_network_name, 
                                  physical_network='exogeni',
                                  vswitch_name=vswitch_name)
                           

# Create the lease
stitch_lease = chi.lease.create_lease(lease_name_stitch, 
                                               start_date=start_date,
                                               end_date=end_date,
                                               reservations=reservation_list)

stitch_lease = chi.lease.wait_for_active(stitch_lease['id'])
    
#Print the lease info
print(json.dumps(stitch_lease, indent=2))

#### Get the Stitched Network Reservation and VLAN

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
stitch_reservation_id = [reservation for reservation in network_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']

print("stitch_reservation_id: " + stitch_reservation_id)

#Get the network
network = chi.network.get_network(stitched_network_name)

#Get the network ID
network_id = network['id']
print(f'Network ID: {network_id}')

#Get the VLAN tag (needed for ExoGENI stitching)
exogeni_vlan = network['provider:segmentation_id']
print(f'exogeni_vlan: {exogeni_vlan}')

## Create the Servers

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=1)

# Create the lease
server_lease = chi.lease.create_lease(lease_name_servers, 
                                      start_date=start_date,
                                      end_date=end_date,
                                      reservations=reservation_list)

chi.lease.wait_for_active(server_lease['id'])
    
#Print the lease info
print(json.dumps(server_lease, indent=2))

In [ ]:
compute_reservation_id = [reservation for reservation in server_lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in server_lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
# Create the server
server = chi.server.create_server(server_name, 
                                  reservation_id=compute_reservation_id,
                                  network_name=network_name, 
                                  image_name=image_name)

chi.server.wait_for_active(server.id)

#### Associate the Floating IP

In [ ]:
floating_ip = chi.server.associate_floating_ip(server.id)

print(f'Floating IP: {floating_ip}')

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit


In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

### Delete Resources

Delete the server.

In [ ]:
chi.server.delete_server(server.id)

#### De-configure Network

In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

#### Release Leases

In [ ]:
chi.lease.delete_lease(network_lease['id'])
chi.lease.delete_lease(server_lease['id'])
chi.lease.delete_lease(stitch_lease['id'])